# Обучение регрессионной модели для предсказания зарплат на hh.ru

Этот ноутбук содержит процесс обучения модели Ridge регрессии с подбором гиперпараметров.


In [ ]:
import numpy as np
import os
from pathlib import Path
from typing import Dict, Any

from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import (
    mean_squared_error,
    mean_absolute_error,
    r2_score,
    explained_variance_score
)
import joblib


## Класс для обучения модели


In [ ]:
class SalaryPredictorTrainer:
    """
    Класс для обучения модели предсказания зарплат.
    
    Выполняет:
    - Загрузку данных из .npy файлов
    - Построение pipeline с нормализацией и регрессией
    - Подбор гиперпараметров через кросс-валидацию
    - Оценку качества модели
    - Сохранение обученной модели
    """
    
    def __init__(
        self,
        x_path: str,
        y_path: str,
        model_save_path: str = "resources/salary_model.pkl",
        n_folds: int = 5,
        random_seed: int = 42,
        n_workers: int = -1
    ):
        self.x_path = x_path
        self.y_path = y_path
        self.model_save_path = model_save_path
        self.n_folds = n_folds
        self.random_seed = random_seed
        self.n_workers = n_workers
        
    def _load_dataset(self) -> tuple[np.ndarray, np.ndarray]:
        """Загружает признаки и целевую переменную из .npy файлов."""
        features = np.load(self.x_path)
        target = np.load(self.y_path)
        # Преобразуем y в одномерный массив, если нужно
        if target.ndim > 1:
            target = target.ravel()
        return features, target
    
    def _create_model_pipeline(self) -> Pipeline:
        """Создает pipeline: нормализация признаков + Ridge регрессия."""
        return Pipeline([
            ("normalizer", StandardScaler()),
            ("regressor", Ridge())
        ])
    
    def _get_hyperparameter_grid(self) -> Dict[str, Any]:
        """Определяет сетку значений alpha для подбора гиперпараметров."""
        alpha_values = np.linspace(0.1, 15.0, 60)  # 60 значений в диапазоне
        return {"regressor__alpha": alpha_values}
    
    def _perform_cross_validation(
        self, 
        X: np.ndarray, 
        y: np.ndarray
    ) -> GridSearchCV:
        """Выполняет кросс-валидацию с подбором гиперпараметров."""
        base_model = self._create_model_pipeline()
        param_grid = self._get_hyperparameter_grid()
        
        cross_validator = KFold(
            n_splits=self.n_folds, 
            shuffle=True, 
            random_state=self.random_seed
        )
        
        metrics = {
            "neg_mean_squared_error": "neg_mean_squared_error",
            "neg_mean_absolute_error": "neg_mean_absolute_error",
            "r2": "r2"
        }
        
        grid_search = GridSearchCV(
            estimator=base_model,
            param_grid=param_grid,
            scoring=metrics,
            refit="neg_mean_squared_error",
            cv=cross_validator,
            n_jobs=self.n_workers,
            return_train_score=True
        )
        
        grid_search.fit(X, y)
        return grid_search
    
    def _compute_metrics(
        self, 
        model: Pipeline, 
        X: np.ndarray, 
        y: np.ndarray
    ) -> Dict[str, float]:
        """Вычисляет метрики качества модели на данных."""
        predictions = model.predict(X)
        
        mse = mean_squared_error(y, predictions)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(y, predictions)
        r2 = r2_score(y, predictions)
        evs = explained_variance_score(y, predictions)
        
        return {
            "mse": mse,
            "rmse": rmse,
            "mae": mae,
            "r2": r2,
            "explained_variance": evs
        }
    
    def _print_results(
        self, 
        grid_search: GridSearchCV, 
        train_metrics: Dict[str, float]
    ) -> None:
        """Выводит результаты обучения в читаемом формате."""
        optimal_alpha = grid_search.best_params_["regressor__alpha"]
        cv_mse = -grid_search.best_score_
        cv_rmse = np.sqrt(cv_mse)
        
        print("\n" + "=" * 75)
        print("РЕЗУЛЬТАТЫ КРОСС-ВАЛИДАЦИИ И ПОДБОРА ГИПЕРПАРАМЕТРОВ".center(75))
        print("=" * 75)
        print(f"Оптимальное значение alpha: {optimal_alpha:.4f}")
        print(f"CV MSE:  {cv_mse:>15.2f}")
        print(f"CV RMSE: {cv_rmse:>15.2f}")
        
        print("\n" + "-" * 75)
        print("МЕТРИКИ НА ОБУЧАЮЩЕЙ ВЫБОРКЕ".center(75))
        print("-" * 75)
        print(f"MSE:                {train_metrics['mse']:>15.2f}")
        print(f"RMSE:               {train_metrics['rmse']:>15.2f}")
        print(f"MAE:                {train_metrics['mae']:>15.2f}")
        print(f"R²:                 {train_metrics['r2']:>15.4f}")
        print(f"Explained Variance: {train_metrics['explained_variance']:>15.4f}")
        print("=" * 75 + "\n")
    
    def _save_model(self, model: Pipeline) -> None:
        """Сохраняет обученную модель в файл."""
        save_dir = Path(self.model_save_path).parent
        save_dir.mkdir(parents=True, exist_ok=True)
        joblib.dump(model, self.model_save_path)
        print(f"Модель сохранена в: {self.model_save_path}")
    
    def train(self) -> Dict[str, Any]:
        """
        Основной метод для запуска процесса обучения.
        
        Returns:
            Словарь с результатами обучения
        """
        print("Загрузка данных...")
        X, y = self._load_dataset()
        print(f"Загружено {X.shape[0]} образцов с {X.shape[1]} признаками")
        
        print("\nЗапуск кросс-валидации и подбора гиперпараметров...")
        grid_search = self._perform_cross_validation(X, y)
        
        best_model = grid_search.best_estimator_
        train_metrics = self._compute_metrics(best_model, X, y)
        
        self._print_results(grid_search, train_metrics)
        
        self._save_model(best_model)
        
        return {
            "grid_search": grid_search,
            "best_model": best_model,
            "train_metrics": train_metrics,
            "best_params": grid_search.best_params_
        }


## Обучение модели


In [ ]:
# Пути к данным
X_FILE = r"c:\Users\vladimir.bezrukov\Downloads\X.npy"
Y_FILE = r"c:\Users\vladimir.bezrukov\Downloads\y.npy"
MODEL_PATH = "resources/salary_model.pkl"

# Создание тренера и обучение
trainer = SalaryPredictorTrainer(
    x_path=X_FILE,
    y_path=Y_FILE,
    model_save_path=MODEL_PATH,
    n_folds=5,
    random_seed=42,
    n_workers=-1
)

results = trainer.train()
